## Worksheet 07
***

In [17]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import coo_matrix
from scipy.sparse.linalg import lsqr

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


To construct X efficiently, all operations with same angle $\alpha$ can be vectorized and all image pixels can be projected on the sensor array at once.

The values, as well as their indices $i$ and $j$, to be put into the sparse matrix, are collected in three corresponding lists. This way a more efficient coordinate format (scipy.sparse.coo_matrix) can be used to assemble the matrix at the end

In [18]:
def constr_X(M, alphas, Np=None):
    """
    Construct design matrix X
    : param M: int 
        Tomogram is an array-like of shape (M, M)
    
    : param alphas: Array-like of shape (n_alphas, )
        List of measurement angles in degrees
        
    : param Np: Default=None
        Optional sensor resolution
    """
    # Define sensor size
    if Np is None:
        Np = int(np.ceil(np.sqrt(2) * M))
        if Np % 2 == 0:
            Np += 1
    # Number of angles
    No = len(alphas)
    # Define design matrix X
    D_beta = M * M
    D_y = No * Np
    
    # Flattened output coordinates
    j = np.mgrid[0 : D_beta].astype(np.int32)
    # Coordinate matrix for the output pixels
    M2 = (M - 1) / 2  # Center matrix
    grid = np.mgrid[-M2 : M - M2, -M2 : M - M2].swapaxes(1, 2).reshape(2, D_beta)
    
    # Collect indices and corresponding values for all iterations
    i_idx = []
    j_idx = []
    weights = []
    
    for k, alpha in enumerate(alphas):
        # Covert angle and projection vector
        alph_rad = np.radians(alpha)
        proj_vec = np.array([np.cos(alph_rad), -np.sin(alph_rad)])
        # Project coordinates
        
        # Compute sensor indices and weights below the projected points
        
        # Rays falling outside the sensor are not counted
        
        # Compute sensor indices and weights above the projected points
        
    # Construct matrix X
    i = np.concatenate(i_idx).astype(np.int32)
    j = np.concatenate(j_idx).astype(np.int32)
    w = np.concatenate(weights)
    X = coo_matrix((w, (i, j)), shape=(D_y, D_beta), dtype=np.float32)
    
    return X